# Design



### Challenge Name: fw_protect (/embsec/design/fw_protect)

The purpose of this tool will be to secure the firmware. Eventually,
you will want to keep the firmware confidential and be able to verify its integrity and authenticity. But for now, 
you will implement a firmware protect tool that does not add security. This tool should simply:

    1. Pack the version "0" into a little-endian short
    2. Load the firmware binary from firmware.bin
    3. Pack the size of the firmware into a little-endian short
    4. Append the release message "First version of firmware." to end of firmware (make sure it is null-terminated)
    5. Build and send a binary blob in the following format to the grader:
    
        [ 0x2 ]      [ 0x2 ]      [ variable ]           [ variable ]
        -------------------------------------------------------------------------
        version | firmware size |   firmware   |   release message + null-byte   




In [1]:
from embsec import Serial
import struct

def fw_protect():
    ser = Serial("/embsec/design/fw_protect")
    # Your code goes here!
    with open('firmware_0.bin', 'rb') as binary_file:
        firmware_update = binary_file.read()
    print("ooooh, we're")
    size = len(firmware_update)
    print("halfway there")
    b = bytes("First version of firmware.\0", 'utf-8')
    
    ser.write(struct.pack(f"<hh{size}s{len(b)}s", 0, size, firmware_update, b))
    print("almost there")
    return ser.read_until()
fw_protect()


ooooh, we're
halfway there
almost there


b'embsec{fw_protect_1bde280499315e29}\n'

### Challenge Name: fw_update (/embsec/design/fw_update)

For this lesson you will implement a basic update tool for the design challenge.
The tool will be used to update the device firmware. You should focus on reading in the firmware blob file, 
and constructing a set of frames to send to the bootloader. Pay attention to the structure of these 
frames, as the bootloader is expecting a certain size and format.

The bootloader is expecting frames of the following format:

      [ 0x2 ]    [ variable ]
    ----------------------------
    frame size |   data... 

Process:

1. Read in binary blob
2. Construct frames from blob
3. Send a 'U' for update mode & receive a confirmation 'U'
4. Send frames sequentially to boot loader



In [42]:
from embsec import Serial
import struct

def fw_update():
    ser = Serial("/embsec/design/fw_update")
    # Your code goes here!
    with open('firmware_1.blob', 'rb') as f_blob:
        blob_contents = f_blob.read()
    
    data_size = len(blob_contents)
    frame_size = 2
    ser.write(b'U')
    print(ser.read(1))
    ser.write(blob_contents[:4])
    ser.read(1)
    print("we've survived", frame_size)
    for i in range(4, data_size, frame_size):
        print("reading", i)
        chunk = blob_contents[i : i+frame_size]
        ser.write(struct.pack(f'>h{frame_size}s', frame_size, chunk))
        print(ser.read(1))
    
    print("finished")
    ser.write(struct.pack(f'>h', 0))
    
    return ser.read_until()
    
fw_update()


bytearray(b'U')
we've survived 2
reading 4
bytearray(b'\x00')
reading 6
bytearray(b'\x00')
reading 8
bytearray(b'\x00')
reading 10
bytearray(b'\x00')
reading 12
bytearray(b'\x00')
reading 14
bytearray(b'\x00')
reading 16
bytearray(b'\x00')
reading 18
bytearray(b'\x00')
reading 20
bytearray(b'\x00')
reading 22
bytearray(b'\x00')
reading 24
bytearray(b'\x00')
reading 26
bytearray(b'\x00')
reading 28
bytearray(b'\x00')
reading 30
bytearray(b'\x00')
reading 32
bytearray(b'\x00')
reading 34
bytearray(b'\x00')
reading 36
bytearray(b'\x00')
reading 38
bytearray(b'\x00')
reading 40
bytearray(b'\x00')
reading 42
bytearray(b'\x00')
reading 44
bytearray(b'\x00')
reading 46
bytearray(b'\x00')
reading 48
bytearray(b'\x00')
reading 50
bytearray(b'\x00')
reading 52
bytearray(b'\x00')
reading 54
bytearray(b'\x00')
reading 56
bytearray(b'\x00')
reading 58
bytearray(b'\x00')
reading 60
bytearray(b'\x00')
reading 62
bytearray(b'\x00')
reading 64
bytearray(b'\x00')
reading 66
bytearray(b'\x00')
reading 68

b'embsec{fw_update_2316313f222d900f}\n'